In [1]:
import torch
import numpy as np
from tqdm import tqdm
import data_utils
import argparse
import pandas as pd
import survface_helper
import sys, os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import net

/home/taes/anaconda3/envs/adaface/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ['CUDA_VISIBLE_DEVICES'] = '1'


In [3]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

In [4]:
def l2_norm(input, axis=1):
    """l2 normalize
    """
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)
    return output, norm


In [5]:
def fuse_features_with_norm(stacked_embeddings, stacked_norms, fusion_method='norm_weighted_avg'):

    assert stacked_embeddings.ndim == 3 # (n_features_to_fuse, batch_size, channel)
    if stacked_norms is not None:
        assert stacked_norms.ndim == 3 # (n_features_to_fuse, batch_size, 1)
    else:
        assert fusion_method not in ['norm_weighted_avg', 'pre_norm_vector_add']

    if fusion_method == 'norm_weighted_avg':
        weights = stacked_norms / stacked_norms.sum(dim=0, keepdim=True)
        fused = (stacked_embeddings * weights).sum(dim=0)
        fused, _ = l2_norm(fused, axis=1)
        fused_norm = stacked_norms.mean(dim=0)
    elif fusion_method == 'pre_norm_vector_add':
        pre_norm_embeddings = stacked_embeddings * stacked_norms
        fused = pre_norm_embeddings.sum(dim=0)
        fused, fused_norm = l2_norm(fused, axis=1)
    elif fusion_method == 'average':
        fused = stacked_embeddings.sum(dim=0)
        fused, _ = l2_norm(fused, axis=1)
        if stacked_norms is None:
            fused_norm = torch.ones((len(fused), 1))
        else:
            fused_norm = stacked_norms.mean(dim=0)
    elif fusion_method == 'concat':
        fused = torch.cat([stacked_embeddings[0], stacked_embeddings[1]], dim=-1)
        if stacked_norms is None:
            fused_norm = torch.ones((len(fused), 1))
        else:
            fused_norm = stacked_norms.mean(dim=0)
    elif fusion_method == 'faceness_score':
        raise ValueError('not implemented yet. please refer to https://github.com/deepinsight/insightface/blob/5d3be6da49275602101ad122601b761e36a66a01/recognition/_evaluation_/ijb/ijb_11.py#L296')
        # note that they do not use normalization afterward.
    else:
        raise ValueError('not a correct fusion method', fusion_method)

    return fused, fused_norm


In [6]:
def infer(model, dataloader, use_flip_test, fusion_method):
    model.eval()
    features = []
    norms = []
    with torch.no_grad():
        for images, idx in tqdm(dataloader):

            feature = model(images.to("cuda:0"))
            if isinstance(feature, tuple):
                feature, norm = feature
            else:
                norm = None

            if use_flip_test:
                fliped_images = torch.flip(images, dims=[3])
                flipped_feature = model(fliped_images.to("cuda:0"))
                if isinstance(flipped_feature, tuple):
                    flipped_feature, flipped_norm = flipped_feature
                else:
                    flipped_norm = None

                stacked_embeddings = torch.stack([feature, flipped_feature], dim=0)
                if norm is not None:
                    stacked_norms = torch.stack([norm, flipped_norm], dim=0)
                else:
                    stacked_norms = None

                fused_feature, fused_norm = fuse_features_with_norm(stacked_embeddings, stacked_norms, fusion_method=fusion_method)
                features.append(fused_feature.cpu().numpy())
                norms.append(fused_norm.cpu().numpy())
            else:
                features.append(feature.cpu().numpy())
                norms.append(norm.cpu().numpy())

    features = np.concatenate(features, axis=0)
    norms = np.concatenate(norms, axis=0)
    return features, norms


In [7]:
def load_pretrained_model(model_name='ir50'):
    # load model and pretrained statedict
    ckpt_path = adaface_models[model_name][0]
    arch = adaface_models[model_name][1]

    model = net.build_model(arch)
    statedict = torch.load(ckpt_path)['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model


In [8]:
model_name = 'ir50'

In [9]:
adaface_models={'ir50': ["../pretrained/ir50_casia_adaface_07131_38ep.ckpt", 'ir_50']}

In [10]:
model = load_pretrained_model(model_name)

In [11]:
model.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [10]:
survface_test = survface_helper.TinyFaceTest(tinyface_root='/home/taes/face_data',
                                                 alignment_dir_name='surv_align')

In [11]:
fusion_method = 'pre_norm_vector_add'

In [12]:
save_path = os.path.join('./survface_result', model_name, "fusion_{}".format(fusion_method))


In [13]:
save_path

'./survface_result/ir50/fusion_pre_norm_vector_add'

In [20]:
if not os.path.exists(save_path):
    os.makedirs(save_path)


In [21]:
print('save_path: {}'.format(save_path))


save_path: ./survface_result/ir50/fusion_pre_norm_vector_add


In [22]:
img_paths = survface_test.image_paths

In [23]:
img_paths

array(['/home/taes/face_data/surv_align/unmated_probe/distractors_22572.jpg',
       '/home/taes/face_data/surv_align/unmated_probe/distractors_2519.jpg',
       '/home/taes/face_data/surv_align/unmated_probe/5737_cam1_86.jpg',
       ...,
       '/home/taes/face_data/surv_align/mated_probe/9448_cam2_119.jpg',
       '/home/taes/face_data/surv_align/mated_probe/9637_cam3_20.jpg',
       '/home/taes/face_data/surv_align/mated_probe/10493_cam3_81.jpg'],
      dtype=object)

In [24]:
print('total images : {}'.format(len(img_paths)))

total images : 242453


In [26]:
batch_size = 32

In [27]:
dataloader = data_utils.prepare_dataloader(img_paths,  batch_size, num_workers=0)

In [31]:
features, norms = infer(model, dataloader, use_flip_test=True, fusion_method=fusion_method)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7577/7577 [14:54<00:00,  8.47it/s]


In [33]:
type(features)

numpy.ndarray

In [16]:
import pickle

In [17]:
# with open('./suv_feature_test.pickle', 'wb') as f:
#     pickle.dump(features, f)

In [18]:
# with open('./suv_norm_test.pickle', 'wb') as f:
#     pickle.dump(norms, f)

In [19]:
with open('suv_feature_test.pickle', 'rb') as f:
    features = pickle.load(f)


In [20]:
with open('suv_norm_test.pickle', 'rb') as f:
    norms = pickle.load(f)


In [12]:
results = survface_test.test_identification(features, ranks=[1,5,20])

NameError: name 'survface_test' is not defined

In [24]:
# def process_chunk(score_mat_chunk, label_mat, ranks, thresholds):
#     """
#     Process a chunk of score_mat using the core logic of DIR_FAR.
#     Return aggregated DIRs for the chunk.
#     """
#     # Place the core logic of DIR_FAR here and adapt as needed to process score_mat_chunk
    
#     # For example:
#     DIRs_chunked = np.zeros([len(thresholds), len(ranks)], dtype=np.float32)
    
#     sort_idx_mat_chunk = np.argsort(score_mat_chunk, axis=1)
#     sorted_label_mat_chunk = np.array([label_mat[i][sort_idx_mat_chunk[i][::-1]] for i in range(label_mat.shape[0])])
    
#     for i, threshold in enumerate(thresholds):
#         for j, rank in enumerate(ranks):
#             success_retrieval = sorted_label_mat_chunk[:, 0:rank].any(axis=1)
#             DIRs_chunked[i, j] += success_retrieval.sum()
    
#     return DIRs_chunked


In [23]:
# def process_chunk(score_mat_chunk, label_mat, ranks, thresholds):
#     """
#     Process a chunk of score_mat using the core logic of DIR_FAR.
#     Return aggregated DIRs for the chunk.
#     """
#     # Place the core logic of DIR_FAR here and adapt as needed to process score_mat_chunk
    
#     # For example:
#     assert score_mat.shape==label_mat.shape
#     # assert np.all(label_mat.astype(np.float32).sum(axis=1) <=1 )
#     # Split the matrix for match probes and non-match probes
#     # subfix _m: match, _nm: non-match
#     # For closed set, we only use the match probes
#     match_indices = label_mat.astype(np.bool).any(axis=1)
#     score_mat_m = score_mat[match_indices,:]
#     label_mat_m = label_mat[match_indices,:]
#     score_mat_nm = score_mat[np.logical_not(match_indices),:]
#     label_mat_nm = label_mat[np.logical_not(match_indices),:]

#     print('mate probes: %d, non mate probes: %d' % (score_mat_m.shape[0], score_mat_nm.shape[0]))

#     # Find the thresholds for different FARs
#     max_score_nm = np.max(score_mat_nm, axis=1)
#     label_temp = np.zeros(max_score_nm.shape, dtype=np.bool)
#     if len(FARs) == 1 and FARs[0] >= 1.0:
#         # If only testing closed-set identification, use the minimum score as threshold
#         # in case there is no non-mate probes
#         thresholds = [np.min(score_mat) - 1e-10]
#         openset = False
#     else:
#         # If there is open-set identification, find the thresholds by FARs.
#         assert score_mat_nm.shape[0] > 0, "For open-set identification (FAR<1.0), there should be at least one non-mate probe!"
#         thresholds = find_thresholds_by_FAR(max_score_nm, label_temp, FARs=FARs)
#         openset = True

#     # Sort the labels row by row according to scores
#     sort_idx_mat_m = np.argsort(score_mat_m, axis=1)
#     sorted_label_mat_m = np.ndarray(label_mat_m.shape, dtype=np.bool)
#     for row in range(label_mat_m.shape[0]):
#         sort_idx = (sort_idx_mat_m[row, :])[::-1]
#         sorted_label_mat_m[row,:] = label_mat_m[row, sort_idx]

#     # Calculate DIRs for different FARs and ranks
#     if openset:
#         gt_score_m = score_mat_m[label_mat_m]
#         assert gt_score_m.size == score_mat_m.shape[0]

#     DIRs = np.zeros([len(FARs), len(ranks)], dtype=np.float32)
#     FARs = np.zeros([len(FARs)], dtype=np.float32)
#     if get_false_indices:
#         false_retrieval = np.zeros([len(FARs), len(ranks), score_mat_m.shape[0]], dtype=np.bool)
#         false_reject = np.zeros([len(FARs), len(ranks), score_mat_m.shape[0]], dtype=np.bool)
#         false_accept = np.zeros([len(FARs), len(ranks), score_mat_nm.shape[0]], dtype=np.bool)
#     for i, threshold in enumerate(thresholds):
#         for j, rank  in enumerate(ranks):
#             success_retrieval = sorted_label_mat_m[:,0:rank].any(axis=1)
#             if openset:
#                 success_threshold = gt_score_m >= threshold
#                 DIRs[i,j] = (success_threshold & success_retrieval).astype(np.float32).mean()
#             else:
#                 DIRs[i,j] = success_retrieval.astype(np.float32).mean()
#             if get_false_indices:
#                 false_retrieval[i,j] = ~success_retrieval
#                 false_accept[i,j] = score_mat_nm.max(1) >= threshold
#                 if openset:
#                     false_reject[i,j] = ~success_threshold
#         if score_mat_nm.shape[0] > 0:
#             FARs[i] = (max_score_nm >= threshold).astype(np.float32).mean()

#     if DIRs.shape[0] == 1 or DIRs.shape[1] == 1:
#         DIRs = DIRs.flatten()

#     if get_false_indices:
#         return DIRs, FARs, thresholds, match_indices, false_retrieval, false_reject, false_accept, sort_idx_mat_m
#     else:
#         return DIRs, FARs, thresholds


In [19]:
# def test_identification(self, features, ranks=[1,5,20], chunk_size=1000):
#     feat_probe = features[self.indices_probe]
#     feat_gallery = features[self.indices_gallery]
    
#     label_mat = self.labels_probe[:, None] == self.labels_gallery[None, :]
    
#     # Determine thresholds and other preprocessing as in your original function
#     # ...

#     total_results = np.zeros([len(thresholds), len(ranks)], dtype=np.float32)
    
#     num_chunks = len(feat_probe) // chunk_size + (len(feat_probe) % chunk_size != 0)
#     for i in range(num_chunks):
#         start_idx = i * chunk_size
#         end_idx = min((i + 1) * chunk_size, len(feat_probe))
        
#         feat_probe_chunk = feat_probe[start_idx:end_idx]
#         score_mat_chunk = np.dot(feat_probe_chunk, feat_gallery.T)
        
#         chunk_results = process_chunk(score_mat_chunk, label_mat[start_idx:end_idx], ranks, thresholds)
        
#         total_results += chunk_results

#     # Average the results if needed
#     total_results /= num_chunks
    
#     return total_results


In [22]:
def test_identification(self, features, ranks=[1,5,20]):
    feat_probe = features[self.indices_probe]
    feat_gallery = features[self.indices_gallery]
    print(feat_probe.shape)
    print(feat_gallery.shape)
    # Use chunked inner product instead of direct matrix multiplication
    return feat_probe,feat_gallery
    score_mat = self.chunked_inner_product(feat_probe, feat_gallery)

    label_mat = self.labels_probe[:,None] == self.labels_gallery[None,:]

    results, _, __ = DIR_FAR(score_mat, label_mat, ranks)

    return results


In [23]:
feat_probe,feat_gallery=test_identification(survface_test,features, ranks=[1,5,20])

(60294, 512)
(182159, 512)


In [26]:
label_mat = survface_test.labels_probe[:,None] == survface_test.labels_gallery[None,:]

In [27]:
label_mat.shape

(60294, 182159)

score mat 60294 * 182159

In [18]:
# results = test_identification(survface_test,features, ranks=[1,5,20])

In [25]:
score_mat=inner_product(feat_probe,feat_gallery)

In [30]:
score_mat.shape

(60294, 182159)

In [26]:
type(score_mat)

numpy.ndarray

In [28]:
score_mat.shape

(60294, 182159)

In [ ]:
with open('./suv_score_mat_test.pickle', 'wb') as f:
    pickle.dump(score_mat, f)

In [29]:
feat_probe.shape

(60294, 512)

In [69]:
1 if feat_probe.shape[0]%10000>0 else 0

1

In [70]:
int(feat_probe.shape[0]/10000) + 1 if feat_probe.shape[0]%10000>0 else 0

7

In [75]:
feat_probe[60000:70000].shape

(294, 512)

In [ ]:
range()

In [72]:
import math

In [76]:
math.ceil(feat_probe.shape[0]/10000)

7

In [73]:
range(0,math.ceil(feat_probe.shape[0]/10000))

7

In [ ]:
feat_probe.shape[0]/10000

In [ ]:
feat_probe

In [ ]:
feat_probe

In [24]:
def inner_product(x1, x2):
    x1, x2 = np.array(x1), np.array(x2)
    if x1.ndim == 3:
        raise ValueError('why?')
        x1, x2 = x1[:,:,0], x2[:,:,0]
    return np.dot(x1, x2.T)


In [30]:
probe_tmp = feat_probe[:10000]

In [31]:
label_mat = survface_test.labels_probe[:10000,None] == survface_test.labels_gallery[None,:]

In [33]:
score_mat = inner_product(probe_tmp,feat_gallery)

In [34]:
score_mat.shape

(10000, 182159)

In [58]:
label_mat = survface_test.labels_probe[:,None] == survface_test.labels_gallery[None,:]

In [ ]:
score_mat = self.inner_product(feat_probe, feat_gallery)

# label_mat = self.labels_probe[:,None] == self.labels_gallery[None,:]


In [28]:
def DIR_FAR(score_mat, label_mat, ranks=[1], FARs=[1.0], get_false_indices=False):
    '''
    Code borrowed from https://github.com/seasonSH/Probabilistic-Face-Embeddings

    Closed/Open-set Identification.
        A general case of Cummulative Match Characteristic (CMC)
        where thresholding is allowed for open-set identification.
    args:
        score_mat:            a P x G matrix, P is number of probes, G is size of gallery
        label_mat:            a P x G matrix, bool
        ranks:                a list of integers
        FARs:                 false alarm rates, if 1.0, closed-set identification (CMC)
        get_false_indices:    not implemented yet
    return:
        DIRs:                 an F x R matrix, F is the number of FARs, R is the number of ranks,
                              flatten into a vector if F=1 or R=1.
        FARs:                 an vector of length = F.
        thredholds:           an vector of length = F.
    '''
    assert score_mat.shape==label_mat.shape
    # assert np.all(label_mat.astype(np.float32).sum(axis=1) <=1 )
    # Split the matrix for match probes and non-match probes
    # subfix _m: match, _nm: non-match
    # For closed set, we only use the match probes
    match_indices = label_mat.astype(np.bool).any(axis=1)
    score_mat_m = score_mat[match_indices,:]
    label_mat_m = label_mat[match_indices,:]
    score_mat_nm = score_mat[np.logical_not(match_indices),:]
    label_mat_nm = label_mat[np.logical_not(match_indices),:]

    print('mate probes: %d, non mate probes: %d' % (score_mat_m.shape[0], score_mat_nm.shape[0]))

    # Find the thresholds for different FARs
    max_score_nm = np.max(score_mat_nm, axis=1)
    label_temp = np.zeros(max_score_nm.shape, dtype=np.bool)
    if len(FARs) == 1 and FARs[0] >= 1.0:
        # If only testing closed-set identification, use the minimum score as threshold
        # in case there is no non-mate probes
        thresholds = [np.min(score_mat) - 1e-10]
        openset = False
    else:
        # If there is open-set identification, find the thresholds by FARs.
        assert score_mat_nm.shape[0] > 0, "For open-set identification (FAR<1.0), there should be at least one non-mate probe!"
        thresholds = find_thresholds_by_FAR(max_score_nm, label_temp, FARs=FARs)
        openset = True

    # Sort the labels row by row according to scores
    sort_idx_mat_m = np.argsort(score_mat_m, axis=1)
    sorted_label_mat_m = np.ndarray(label_mat_m.shape, dtype=np.bool)
    for row in range(label_mat_m.shape[0]):
        sort_idx = (sort_idx_mat_m[row, :])[::-1]
        sorted_label_mat_m[row,:] = label_mat_m[row, sort_idx]

    # Calculate DIRs for different FARs and ranks
    if openset:
        gt_score_m = score_mat_m[label_mat_m]
        assert gt_score_m.size == score_mat_m.shape[0]

    DIRs = np.zeros([len(FARs), len(ranks)], dtype=np.float32)
    FARs = np.zeros([len(FARs)], dtype=np.float32)
    if get_false_indices:
        false_retrieval = np.zeros([len(FARs), len(ranks), score_mat_m.shape[0]], dtype=np.bool)
        false_reject = np.zeros([len(FARs), len(ranks), score_mat_m.shape[0]], dtype=np.bool)
        false_accept = np.zeros([len(FARs), len(ranks), score_mat_nm.shape[0]], dtype=np.bool)
    for i, threshold in enumerate(thresholds):
        for j, rank  in enumerate(ranks):
            success_retrieval = sorted_label_mat_m[:,0:rank].any(axis=1)
            if openset:
                success_threshold = gt_score_m >= threshold
                DIRs[i,j] = (success_threshold & success_retrieval).astype(np.float32)
            else:
                DIRs[i,j] = success_retrieval.astype(np.float32)
            if get_false_indices:
                false_retrieval[i,j] = ~success_retrieval
                false_accept[i,j] = score_mat_nm.max(1) >= threshold
                if openset:
                    false_reject[i,j] = ~success_threshold
        if score_mat_nm.shape[0] > 0:
            FARs[i] = (max_score_nm >= threshold).astype(np.float32)

    if DIRs.shape[0] == 1 or DIRs.shape[1] == 1:
        DIRs = DIRs.flatten()

    if get_false_indices:
        return DIRs, FARs, thresholds, match_indices, false_retrieval, false_reject, false_accept, sort_idx_mat_m
    else:
        return DIRs, FARs, thresholds


In [40]:
match_indices = label_mat.astype(np.bool).any(axis=1)
score_mat_m = score_mat[match_indices,:]
label_mat_m = label_mat[match_indices,:]
score_mat_nm = score_mat[np.logical_not(match_indices),:]
label_mat_nm = label_mat[np.logical_not(match_indices),:]


/home/taes/anaconda3/envs/adaface/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [41]:
ranks=[1,5,20]
FARs=[1.0]

In [42]:
max_score_nm = np.max(score_mat_nm, axis=1)
label_temp = np.zeros(max_score_nm.shape, dtype=np.bool)
if len(FARs) == 1 and FARs[0] >= 1.0:
    # If only testing closed-set identification, use the minimum score as threshold
    # in case there is no non-mate probes
    thresholds = [np.min(score_mat) - 1e-10]
    openset = False
else:
    # If there is open-set identification, find the thresholds by FARs.
    assert score_mat_nm.shape[0] > 0, "For open-set identification (FAR<1.0), there should be at least one non-mate probe!"
    thresholds = find_thresholds_by_FAR(max_score_nm, label_temp, FARs=FARs)
    openset = True


/home/taes/anaconda3/envs/adaface/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [43]:
    sort_idx_mat_m = np.argsort(score_mat_m, axis=1)
    sorted_label_mat_m = np.ndarray(label_mat_m.shape, dtype=np.bool)
    for row in range(label_mat_m.shape[0]):
        sort_idx = (sort_idx_mat_m[row, :])[::-1]
        sorted_label_mat_m[row,:] = label_mat_m[row, sort_idx]

    # Calculate DIRs for different FARs and ranks
    if openset:
        gt_score_m = score_mat_m[label_mat_m]
        assert gt_score_m.size == score_mat_m.shape[0]

    DIRs = np.zeros([len(FARs), len(ranks)], dtype=np.float32)
    FARs = np.zeros([len(FARs)], dtype=np.float32)


/home/taes/anaconda3/envs/adaface/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
for i, threshold in enumerate(thresholds):
    for j, rank  in enumerate(ranks):
        success_retrieval = sorted_label_mat_m[:,0:rank].any(axis=1)
        if openset:
            success_threshold = gt_score_m >= threshold
            DIRs[i,j] = (success_threshold & success_retrieval).astype(np.float32)
        else:
            DIRs[i,j] = success_retrieval.astype(np.float32)
        if get_false_indices:
            false_retrieval[i,j] = ~success_retrieval
            false_accept[i,j] = score_mat_nm.max(1) >= threshold
            if openset:
                false_reject[i,j] = ~success_threshold
    if score_mat_nm.shape[0] > 0:
        FARs[i] = (max_score_nm >= threshold).astype(np.float32)


In [44]:
success_retrieval = sorted_label_mat_m[:,0:1].any(axis=1)

In [45]:
sorted_label_mat_m.shape

(10000, 182159)

In [77]:
success_retrieval.shape

(10000,)

In [91]:
sorted_label_mat_m[:,0:5].any(axis=1).sum()

622

In [92]:
sorted_label_mat_m[:,0:20].any(axis=1).sum()

1070

In [ ]:
np.zeros([len(FARs), len(sorted_label_mat_m[:,0:5].any(axis=1).shape)], dtype=np.float32)

In [87]:
sorted_label_mat_m[:,0:5].any(axis=1).shape

(10000,)

In [88]:
sorted_label_mat_m[:,0:5].any(axis=1).mean()

0.0622

In [93]:
DIRs = np.zeros([len(FARs), 3], dtype=np.float32)

In [101]:
(DIRs + np.ones([len(FARs), 3], dtype=np.float32))/2

array([[0.5, 0.5, 0.5]], dtype=float32)

In [90]:
DIRs.flatten().shape

(3,)

In [84]:
np.concatenate([DIRs,np.zeros([len(FARs), len(ranks)], dtype=np.float32)]).shape

(2, 3)

In [49]:
success_retrieval.sum()

322

NameError: name 'threshold' is not defined

In [ ]:
60294, 512

In [36]:
score_mat.shape

(10000, 182159)

In [39]:
label_mat.shape

(10000, 182159)

In [51]:
thresholds = [np.min(score_mat) - 1e-10]

In [55]:
(max_score_nm >= thresholds[0]).astype(np.float32)

array([], dtype=float32)

In [59]:
score_mat_nm = score_mat[np.logical_not(match_indices),:]

In [60]:
score_mat_nm

array([], shape=(0, 182159), dtype=float32)

In [58]:
score_mat_nm

array([], shape=(0, 182159), dtype=float32)

In [56]:
max_score_nm = np.max(score_mat_nm, axis=1)

In [57]:
max_score_nm

array([], dtype=float32)

In [ ]:
results, _, __ = DIR_FAR(score_mat, label_mat, [1,5,20])

/home/taes/anaconda3/envs/adaface/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
